validation series 1 is created for context type

In [14]:
import os
import soundfile as sf
import time
import math
import numpy as np
from natsort import natsorted
import soundfile as sf
import librosa
import h5py
from sklearn import preprocessing

In [34]:
file_series = str(1)
addr_X_file = 'TSP/Predicted/Valid_predicted_TSP_FFT' + file_series + '.hdf5'
# addr_y_file = 'TSP/Organized/concatenated/Valid_TSP_clean_FFTs'  + file_series + '.hdf5'

In [35]:
a = h5py.File('TSP/Organized/concatenated/valid_all_len'  + file_series + '.hdf5','r')
all_len = a['valid_all_len'][:]
print(all_len)
a.close()

[    0   219   395   584   803   993  1174  1360  1550  1739  1928  2104
  2290  2471  2687  2876  3092  3278  3497  3713  3889  4105  4295  4481
  4657  4846  5032  5222  5441  5657  5838  6019  6195  6385  6566  6785
  6961  7188  7405  7615  7822  8025  8201  8377  8580  8783  9006  9216
  9446  9649  9876 10086 10262 10479 10655 10862 11072 11299 11475 11678
 11895 12112 12331 12567 12774 12993 13221 13428 13604 13811 14028 14251
 14461 14677 14866 15052 15233 15414 15604 15793 15983 16169 16350 16540
 16759 16949 17168 17384 17573 17759 17945 18126 18342 18532 18713 18932
 19113 19329 19520 19709 19928 20118 20309 20495 20684 20903 21119 21305
 21494]


In [36]:
X_file = h5py.File(addr_X_file,'r')
X = X_file['Valid_predicted_TSP_FFT' + file_series][:]
print(X.shape)

(21494, 2050)


# Functions

In [54]:
flag = np.asarray(all_len)
window_size = 2
r = []
r0 = []
r1 = []
half_len = X.shape[1]//2
for I in range(len(flag)-1):
    X = preprocessing.normalize(X, norm='l2', axis=0, copy=True)
    
    temp0 = X[flag[I]:flag[I+1],0:half_len]
    temp0 = np.concatenate((temp0,np.repeat(np.reshape(temp0[-1,:],(1,temp0.shape[1])),[window_size],axis=0)), axis=0)
    temp0 = np.concatenate((np.repeat(np.reshape(temp0[0,:],(1,temp0.shape[1])),[window_size],axis=0),temp0), axis=0)
    
    for i in range(0,temp0.shape[0]-2*window_size):
        r0.append(np.ndarray.flatten(temp0[i:i+window_size*2+1,:]))


    temp1 = X[flag[I]:flag[I+1],half_len:]
    temp1 = np.concatenate((temp1,np.repeat(np.reshape(temp1[-1,:],(1,temp1.shape[1])),[window_size],axis=0)), axis=0)
    temp1 = np.concatenate((np.repeat(np.reshape(temp1[0,:],(1,temp1.shape[1])),[window_size],axis=0),temp1), axis=0)
    
    for i in range(0,temp1.shape[0]-2*window_size):
        r1.append(np.ndarray.flatten(temp1[i:i+window_size*2+1,:]))

#     print(np.asarray(r0).shape)
#     print(np.asarray(r1).shape)
    r = np.concatenate((np.asarray(r0),np.asarray(r1)),axis=1)
#     print(r.shape)



In [55]:
X.shape

(21494, 2050)

In [56]:
np.asarray(r).shape

(21494, 10250)

In [53]:
with h5py.File('TSP/Predicted/Valid_predicted_TSP_FFT_context' + file_series + '.hdf5', 'w') as h5f:
    h5f.create_dataset('Valid_predicted_TSP_FFT_context'  + file_series, data=r)
    h5f.close()